In [51]:
# %%
from random import randint, uniform
from numpy import *

# %%
def add_two(mat):
    a = randint(0, len(mat)-1) # Choose any x
    b = randint(0, len(mat)-1) # Choose any y

    while(mat[a][b]!=0):  # If the x,y coord is occupied, try again and again
        a = randint(0,len(mat)-1)
        b = randint(0,len(mat)-1)

    # 10% chance that the number will be 4 instead of 2. 
    mat[a][b] = (uniform(0, 1) < 0.1)*2 + 2  

    return mat

def reverse(mat):
    new=[]
    for i in range(len(mat)):
        new.append([])
        for j in range(len(mat[0])):
            new[i].append(mat[i][len(mat[0])-j-1])
    return new

def transpose(mat):
    new=[]
    for i in range(len(mat[0])):
        new.append([])
        for j in range(len(mat)):
            new[i].append(mat[j][i])
    return new

def cover_up(mat):
    new = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
    done = False
    for i in range(4):
        count=0
        for j in range(4):
            if mat[i][j]!=0:
                new[i][count]=mat[i][j]
                if j!=count:
                    done=True
                count+=1
    return new, done

def merge(mat):
    done = False
    addPoints = 0
    for i in range(4):
         for j in range(3):
             if mat[i][j]==mat[i][j+1] and mat[i][j]!=0:
                 mat[i][j]*=2
                 addPoints += mat[i][j]
                 mat[i][j+1]=0
                 done=True
    return mat, done, addPoints

In [52]:
# %%
class GameState:
    """ Simple version of the 2048 board with moves, processing, and point system available.
        Use this class to pair with MCTS algorithm. """

    def __init__(self, mat):
        self.matrix = mat
        self.point_count = 0

    def __str__(self):
        s = ""
        for i in range(4):
            for j in range(4):
                s += "\t"
                s += str(self.matrix[i][j])
                if j == 3:
                    s += "\n"
        return s

    def game_state(self):
        mat = self.matrix
        for i in range(len(mat)):
            for j in range(len(mat[0])):
                if mat[i][j]==2048:
                    return 'win'
        for i in range(len(mat)-1): # Intentionally reduced to check the row on the right and below
            for j in range(len(mat[0])-1): # More elegant to use exceptions but most likely this will be their solution
                if mat[i][j]==mat[i+1][j] or mat[i][j+1]==mat[i][j]:
                    return 'not over'
        for i in range(len(mat)): # Check for any zero entries
            for j in range(len(mat[0])):
                if mat[i][j]==0:
                    return 'not over'
        for k in range(len(mat)-1): # To check the left/right entries on the last row
            if mat[len(mat)-1][k]==mat[len(mat)-1][k+1]:
                return 'not over'
        for j in range(len(mat)-1): # Check up/down entries on last column
            if mat[j][len(mat)-1]==mat[j+1][len(mat)-1]:
                return 'not over'
        return 'lose'

    # Done indicates whether the move does anything
    def up(self):
        game = self.matrix
        game = transpose(game)
        game, done = cover_up(game)
        temp = merge(game)
        game = temp[0]
        done = done or temp[1]
        game = cover_up(game)[0]
        game = transpose(game)
        add_points = temp[2]
        return game, done, add_points

    def down(self):
        game = self.matrix
        game = reverse(transpose(game))
        game, done = cover_up(game)
        temp = merge(game)
        game = temp[0]
        done = done or temp[1]
        game = cover_up(game)[0]
        game = transpose(reverse(game))
        add_points = temp[2]
        return game, done, add_points

    def left(self):
        game = self.matrix
        game, done = cover_up(game)
        temp = merge(game)
        game = temp[0]
        done = done or temp[1]
        game = cover_up(game)[0]
        add_points = temp[2]
        return game, done, add_points

    def right(self):
        game = self.matrix
        game = reverse(game)
        game, done = cover_up(game)
        temp = merge(game)
        game = temp[0]
        done = done or temp[1]
        game = cover_up(game)[0]
        game = reverse(game)
        add_points = temp[2]
        return game, done, add_points

    def clone(self):
        st = GameState(self.matrix)
        st.point_count = self.point_count
        return st
    
    def do_move(self, move):
        """ Move input should be one of the following: "up", "down", "left", "right"
            Make sure when this function is called, the move is a possible move. """
        move_funcs = {
            'up':       self.up(),
            'down':     self.down(),
            'left':     self.left(),
            'right':    self.right()
        }

        self.matrix, _, add_points = move_funcs[move]
        self.point_count += add_points # Update Points
        
        # Check if there any zeros in the grid.
        zero_exists = False
        for i in range(4):
            for j in range(4):
                if self.matrix[i][j] == 0:
                    zero_exists = True
                    break
        
        if zero_exists:
            self.matrix = add_two(self.matrix) # Add 2 or 4 in the matrix
            game_over = self.game_state() != 'not over'
            return game_over 
        else:
            game_over = True
            return game_over 
        
    def get_moves(self):
        """ Get all possible moves from this state. """
        _, done_up, _     = self.up()
        _, done_down, _   = self.down()
        _, done_left, _   = self.left()
        _, done_right, _  = self.right()

        move_possible = []
        if done_up:
            move_possible.append("up")
        if done_down:
            move_possible.append("down")
        if done_left:
            move_possible.append("left")
        if done_right:
            move_possible.append("right")
        
        return move_possible

    def get_result(self):
        """ Get the score of the given state."""
        return self.point_count

# %%
if __name__ == "__main__":
    initialMat = [[4,2,4,0],[0,0,4,0],[0,2,0,0],[8,0,2,0]]
    deathMatrix = [[8, 4, 2, 8], [2, 16, 8, 4], [256, 32, 4, 2], [4, 2, 4, 2]]

In [66]:
# Random agent

from random import randrange

mat = GameState([[0,0,0,0],[0,2,0,0],[0,0,0,0],[0,0,2,0]])
while(1):
    x = mat.get_moves()
    if len(x)==0:
        break
    i = randint(0,len(x)-1)
    if len(x)==0:
        break
    mat.do_move(x[i])
print(mat)

	2	8	4	2
	32	64	8	4
	16	2	32	16
	2	4	8	2



In [ ]:
# Random agent with foresight

num_trials = 100
stats1 = []

for j in range(num_trials):
    mat = GameState([[0,0,0,0],[0,2,0,0],[0,0,0,0],[0,0,2,0]])
    while(1):
        i = randrange(len(mat.get_moves()))
        mat.do_move(mat.get_moves()[i])
        if mat.game_state()!='not over':
            break
    stats1.append(mat.get_result())
    if j == 0:
        bestmat = mat
    if bestmat.get_result() < mat.get_result():
        bestmat = mat
print(max(stats1))
print(bestmat)

2088
	4	2	16	4
	2	4	256	8
	8	16	4	16
	2	4	32	4



In [ ]:
# Random agent with more sampling

bestmove = ''
main_game = GameState([[0,0,0,0],[0,2,0,0],[0,0,0,0],[0,0,2,0]])
while(1):
    counter = 1
    mat = main_game
    count = 1
    for j in range(10000):
        while(1):
            if len(mat.get_moves())==0:
                break
            i = randrange(len(mat.get_moves())) # random move assigned
            if counter == 1:
                action = mat.get_moves()[i] # Action copied 
                bestmat = mat
                counter = 0
            mat.do_move(mat.get_moves()[i])
            # if mat.game_state()!='not over':
            #     break
            if count > 20000 or mat.game_state()!='not over':
                break
            count = count + 1
            if bestmat.get_result() < mat.get_result():
                bestmat = mat
                bestmove = action
    if len(mat.get_moves())==0:
        break
    main_game.do_move(bestmove)
    mat = main_game
    if main_game.game_state()!='not over':
        break

In [55]:
print(main_game)

	4	8	4	2
	16	4	32	4
	2	8	128	8
	8	2	16	2



In [56]:
# Random agent with heuristic learning

from random import choice

def evaluate_board(state):
    grid = state.matrix
    # Encourage higher number of empty tiles 
    empty_tiles = sum(row.count(0) for row in grid)
    # Encourage high corner values
    corner_bonus = grid[0][0] + grid[0][3] + grid[3][0] + grid[3][3]
    # Encourage monotonicity 
    monotonicity = sum(grid[i][j] <= grid[i][j+1] for i in range(4) for j in range(3)) + \
                   sum(grid[i][j] <= grid[i+1][j] for i in range(3) for j in range(4))
    return (empty_tiles * 5) + (corner_bonus * 20) + (monotonicity * 2) # WEIGHTS TO BE ADJUSTED

def get_best_move(state, num_simulations=100):
    moves = state.get_moves()
    if not moves:
        return None
    best_move = None
    best_score = float('-inf')
    for move in moves:
        total_score = 0
        for _ in range(num_simulations):
            sim_state = state.clone()
            sim_state.do_move(move)
            for _ in range(50):
                possible_moves = sim_state.get_moves()
                if not possible_moves:
                    break
                sim_state.do_move(choice(possible_moves))
            total_score += evaluate_board(sim_state)
        avg_score = total_score / num_simulations
        if avg_score > best_score:
            best_score = avg_score
            best_move = move
    return best_move
game = GameState([[0,0,0,0], [0,2,0,0], [0,0,0,0], [0,0,2,0]])
best_moves = [] 
while game.game_state() == 'not over':
    best_move = get_best_move(game, num_simulations=100)
    if best_move:
        best_moves.append(best_move) 
        game.do_move(best_move)
    else:
        break 
print("Final Score:", game.get_result())
print("Final Board:")
print(game.matrix)
print("Best moves noted:", best_moves) 

C:\Users\maadh\AppData\Local\Temp\ipykernel_27636\1286346893.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  empty_tiles = sum(row.count(0) for row in grid)
C:\Users\maadh\AppData\Local\Temp\ipykernel_27636\1286346893.py:12: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  monotonicity = sum(grid[i][j] <= grid[i][j+1] for i in range(4) for j in range(3)) + \
C:\Users\maadh\AppData\Local\Temp\ipykernel_27636\1286346893.py:13: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum(grid[i][j] <= grid[i+1][j] for i in range(3) for j in range(4))


Final Score: 1300
Final Board:
[[2, 4, 2, 8], [4, 16, 64, 2], [8, 32, 2, 16], [128, 2, 4, 8]]
Best moves noted: ['right', 'left', 'down', 'up', 'left', 'left', 'left', 'right', 'right', 'down', 'up', 'down', 'left', 'left', 'up', 'right', 'down', 'up', 'up', 'left', 'up', 'left', 'left', 'down', 'right', 'left', 'down', 'left', 'up', 'down', 'left', 'up', 'left', 'left', 'left', 'down', 'right', 'left', 'right', 'down', 'down', 'up', 'down', 'left', 'right', 'down', 'right', 'left', 'left', 'up', 'left', 'down', 'down', 'up', 'right', 'down', 'left', 'up', 'down', 'down', 'up', 'down', 'up', 'right', 'up', 'left', 'up', 'right', 'down', 'up', 'down', 'left', 'up', 'left', 'right', 'up', 'down', 'left', 'up', 'left', 'right', 'up', 'left', 'down', 'up', 'down', 'left', 'up', 'left', 'up', 'left', 'left', 'right', 'up', 'up', 'left', 'up', 'left', 'down', 'right', 'up', 'down', 'up', 'down', 'left', 'right', 'left', 'right', 'down', 'left', 'down', 'up', 'right', 'left', 'right', 'left',

In [57]:
# Best sequence learning

# from itertools import permutations

# def cycle_moves(game_state, max_moves=10):
#     moves = ["up", "right", "down", "left"]
#     best_permutation = None
#     max_score = -1
    
#     all_permutations = list(permutations(moves))
    
#     for perm in all_permutations:
#         test_state = game_state.clone()
#         score = 0
#         move_count = 0
        
#         while move_count < max_moves:
#             for move in perm:
#                 if move in test_state.get_moves():
#                     test_state.do_move(move)
#                     score = sum(sum(row) for row in test_state.matrix)
#                     move_count += 1
#                     if move_count >= max_moves:
#                         break
#                 else:
#                     break
        
#         if score > max_score:
#             max_score = score
#             best_permutation = perm
    
#     return best_permutation

# def policy_function(game_state):
#     best_perm = cycle_moves(game_state)
#     return best_perm[0] if best_perm else None


In [58]:
# initialMat = [[4,2,4,0],[0,0,4,0],[0,2,0,0],[8,0,2,0]]
# game = GameState(initialMat)
# best_move = policy_function(game)
# print(f"Best move based on simulation: {best_move}")


In [81]:
# Permutational Heuristic

initialMat = [[0,0,0,0],[0,2,0,0],[0,0,0,0],[0,0,2,0]]
game = GameState(initialMat)

while(1):
    nmoves = game.get_moves()
    if len(nmoves)==0:
        break
    if 'left' in nmoves:
        game.do_move('left')
        continue
    if 'down' in nmoves:
        game.do_move('down')
        continue
    x = randint(0,len(nmoves)-1)
    game.do_move(game.get_moves()[x])
print(game)    

	2	16	8	2
	4	32	16	4
	8	64	32	16
	256	128	4	2

